In [ ]:
from diagrams import Cluster, Diagram, Edge
from diagrams.aws import compute, database, enduser, integration, network, security, storage
from diagrams.custom import Custom
from diagrams.generic import storage as generic_storage
from diagrams.onprem import client
from diagrams. onprem import network as onprem_network
from diagrams.programming import flowchart, framework, language


show_diagrams = True

def display_diagram(diag):
    if show_diagrams:
        display(diag)

In [ ]:
graph_attr = {
    "layout": "dot",
    "compound": "true",
}

transparent_bg = {
    "label":"x",
    "bgcolor":"transparent",
    # "penwidth":"0.0"    
}

In [ ]:
with Diagram(filename="workspaces", outformat="png", show=False, direction="LR", graph_attr=graph_attr) as diag:
    with Cluster("AWS TRE Account", graph_attr=transparent_bg):
        with Cluster("Project A S3"):
            projecta = [
                storage.S3("Study A"),
                storage.SimpleStorageServiceS3Bucket("User-1 files"),
            ]
        with Cluster("Project B S3"):
            projectb = [
                storage.S3("Study B"),
                storage.SimpleStorageServiceS3Bucket("User-1 files"),
            ]

    with Cluster("AWS Researchers Account", graph_attr=transparent_bg):
        with Cluster("Secure compute A"):
            windows = compute.EC2Instance("Windows")
            rdp = enduser.Appstream20("RDP")
        with Cluster("Secure compute B"):
            hpc = compute.Batch("HPC")
            ubuntu = compute.EC2("Ubuntu")
            vnc = enduser.Appstream20("VNC")

    projecta[1] - windows << rdp
    [projectb[0] - hpc, projectb[1]] - ubuntu << vnc
    [rdp, vnc] << client.User("User-1")

display_diagram(diag)

In [ ]:
with Diagram(filename="researcher-network", outformat="png", show=False, direction="LR", graph_attr=graph_attr) as diag:

    with Cluster("Shared services"):
        services = [
            compute.EC2Instance("License server"),
            compute.EC2Instance("Anti-virus proxy"),
            compute.EC2Instance("Package proxy"),
        ]
        network_services = network.VPC("Services network")
        gateway = network.NATGateway("NAT Gateway")
        services - network_services - gateway

    gateway - onprem_network.Internet("")
    peer = network.VPCPeering("")

    with Cluster("Project workspace"):
        u1b = compute.EC2Instance("User 1")
        sb = storage.S3("Data B")
        u1b - sb
        network_services - peer - u1b
    # u1a - Edge(taillabel="X",style="dotted",fontcolor="red",labeldistance="3",labelangle="0") - u1b

display_diagram(diag)

## Researcher web UI

In [ ]:
ga = graph_attr.copy()
#ga["size"] = "100,!10"
# ga["layout"] = "neato"

with Diagram(filename="researcher-webui", outformat="png", show=False,
             direction="LR", graph_attr=ga) as diag:

    with Cluster("TRE application"):
        kc = Custom("Keycloak identity", "./keycloak-logo.png")
        cog = security.Cognito("Cognito")
        react = framework.React("React front-end")
        db = database.Dynamodb("DynamoDB")
        logic = integration.StepFunctions("Step functions/Lambda")

    kc - cog
    db - logic

    researcher = client.User("Researcher")
    logic - react - researcher

display_diagram(diag)

## TRE researcher workspace diagram

In [ ]:
ga = graph_attr.copy()
#ga["size"] = "100,!10"
#ga["layout"] = "neato"

with Diagram(filename="researcher-workspace", outformat="png", show=False,
             direction="LR", graph_attr=ga) as diag:
    #, graph_attr=transparent_bg

    with Cluster("AWS workspace account"):
        s_rw = storage.S3("User read-write")
        with Cluster("CloudFormation stack"):
            w = compute.EC2Instance("Windows EC2")
            appstream = enduser.Appstream20("Appstream RDP")
        
    with Cluster("AWS Data account"):
        s_raw = storage.S3("Sensitive data")

    # client.User("Data Manager") >> s_raw
    (s_raw, s_rw) - w << appstream << Edge(label="Virtual desktop only", minlen="2") >> client.User("Researcher")
 
display_diagram(diag)